# P4Lang Tutorials of FABRIC

This notebook walks the user through setting up a FABRIC eperiment that is suitiable for completing the P4 tutorials created by [P4Lang](https://github.com/p4lang/tutorials). The tutorials were origianlly designed to use a mininet topology. This example replaces the mininet topology with a FABRIC experiemnt topology that may span multiple sites across the FABRIC testbed.

Additional resources:
- [FABRIC Knowledge Base](https://learn.fabric-testbed.net/)
- [FABRIC Forums](https://learn.fabric-testbed.net/forums/)
- [P4Lang Tutorials](https://github.com/p4lang/tutorials)
- [P4Lang YouTube Presentations](https://www.youtube.com/channel/UCOQAFkDKucJWr-KafdJsdIQ)

In [1]:
import os

os.environ['FABRIC_CREDMGR_HOST']='cm.fabric-testbed.net'
os.environ['FABRIC_ORCHESTRATOR_HOST']='orchestrator.fabric-testbed.net'
os.environ['FABRIC_TOKEN_LOCATION']=os.environ['HOME']+'/work/fabric_token.json'

os.environ['FABRIC_BASTION_USERNAME']='pruth'
os.environ['FABRIC_BASTION_KEY_LOCATION']=os.environ['HOME']+'/work/.ssh/id_rsa_fabric'

os.environ['FABRIC_SLICE_PRIVATE_KEY_FILE']=os.environ['HOME']+'/.ssh/id_rsa'
os.environ['FABRIC_SLICE_PUBLIC_KEY_FILE']=os.environ['HOME']+'/.ssh/id_rsa.pub'

os.environ['FABRIC_BASTION_HOST'] = 'bastion-1.fabric-testbed.net'
os.environ['FABRIC_BASTION_HOST_PRIVATE_IPV4'] = '192.168.11.226'
os.environ['FABRIC_BASTION_HOST_PRIVATE_IPV6'] = '2600:2701:5000:a902::c'

## Basic FABRIC Slice Configuration

In [2]:
import json
import traceback
from fabrictestbed_extensions.fablib.fablib import fablib

## Configure Slice Parameters

This section builds the experiment slice 

<img src="figs/fabric_slice.png" width="800"/>



In [3]:
# Slice 
slice_name = 'P4Lang_Tutorial1'

# Switches
s1_name = "s1"
s2_name = "s2"
s3_name = "s3"

switch_cores = 2
switch_ram = 8
switch_disk = 40

# Hosts
h1_name = "h1"
h2_name = "h2"
h3_name = "h3"

h1_ip="10.0.1.1"
h1_ip="10.0.2.2"
h1_ip="10.0.3.3"

host_cores = 2
host_ram = 8
host_disk = 10
# Sites
site_1 = 'MAX'
site_2 = 'TACC'
site_3 = 'UTAH'

# All node properties
#username = 'ubuntu'
image = 'default_ubuntu_20'
vlan = '1000'
#image_type = 'qcow2'



### Create the Slice

In [4]:
try:
    #Create Slice
    slice = fablib.new_slice(name=slice_name)
    
    # Add switch node s1
    s1 = slice.add_node(name=s1_name, site=site_1)
    s1.set_capacities(cores=switch_cores, ram=switch_ram, disk=switch_disk)
    s1.set_image(image)
    [s1_iface_local] = s1.add_component(model='NIC_Basic', name="s1_local_nic").get_interfaces()
    [s1_iface_to_s2, s1_iface_to_s3] = s1.add_component(model='NIC_ConnectX_5', name="s1_switch_nic").get_interfaces()
    s1_iface_to_s2.set_vlan(vlan=vlan)
    s1_iface_to_s3.set_vlan(vlan=vlan)

    # Add switch node s2
    s2 = slice.add_node(name=s2_name, site=site_2)
    s2.set_capacities(cores=switch_cores, ram=switch_ram, disk=switch_disk)
    s2.set_image(image)
    [s2_iface_local] = s2.add_component(model='NIC_Basic', name="s2_local_nic").get_interfaces()
    [s2_iface_to_s1, s2_iface_to_s3] = s2.add_component(model='NIC_ConnectX_5', name="s2_switch_nic").get_interfaces()
    s2_iface_to_s1.set_vlan(vlan=vlan)
    s2_iface_to_s3.set_vlan(vlan=vlan)
   
    # Add switch node s3
    s3 = slice.add_node(name=s3_name, site=site_3)
    s3.set_capacities(cores=switch_cores, ram=switch_ram, disk=switch_disk)
    s3.set_image(image)
    [s3_iface_local] = s3.add_component(model='NIC_Basic', name="s3_local_nic").get_interfaces()
    [s3_iface_to_s1, s3_iface_to_s2] = s3.add_component(model='NIC_ConnectX_5', name="s3_switch_nic").get_interfaces()
    s3_iface_to_s1.set_vlan(vlan=vlan)
    s3_iface_to_s2.set_vlan(vlan=vlan)
    
    # Add host node h1
    h1 = slice.add_node(name=h1_name, site=site_1)
    h1.set_capacities(cores=host_cores, ram=host_ram, disk=host_disk)
    h1.set_image(image)
    [h1_iface] = h1.add_component(model='NIC_Basic', name="h1_nic").get_interfaces()
    
    # Add host node h2
    h2 = slice.add_node(name=h2_name, site=site_2)
    h2.set_capacities(cores=host_cores, ram=host_ram, disk=host_disk)
    h2.set_image(image)
    [h2_iface] = h2.add_component(model='NIC_Basic', name="h2_nic").get_interfaces()
 
    # Add host node h3
    h3 = slice.add_node(name=h3_name, site=site_3)
    h3.set_capacities(cores=host_cores, ram=host_ram, disk=host_disk)
    h3.set_image(image)
    [h3_iface] = h3.add_component(model='NIC_Basic', name="h3_nic").get_interfaces()
    
    #Add swtich networks
    switch_net1 = slice.add_l2network(name="net_s1_s2", interfaces=[s1_iface_to_s2, s2_iface_to_s1])
    swtich_net2 = slice.add_l2network(name="net_s2_s3", interfaces=[s2_iface_to_s3, s3_iface_to_s2])
    swtich_net3 = slice.add_l2network(name="net_s3_s1", interfaces=[s3_iface_to_s1, s1_iface_to_s3])

    #Add host networks 
    host_net1 = slice.add_l2network(name="host_net1", interfaces=[s1_iface_local, h1_iface])
    host_net2 = slice.add_l2network(name="host_net2", interfaces=[s2_iface_local, h2_iface])
    host_net3 = slice.add_l2network(name="host_net3", interfaces=[s3_iface_local, h3_iface])    
    
    #Submit Slice Request
    slice.submit(wait_progress=True)
except Exception as e:
    print(f"Error: {e}")
    traceback.print_exc()
    

Waiting for slice .............. Slice state: StableOK


## Get the Slice

In [5]:
try:
    slice = fablib.get_slice(name=slice_name)
    for node in slice.get_nodes():
        print("Node:")
        print(f"   Name              : {node.get_name()}")
        print(f"   Cores             : {node.get_cores()}")
        print(f"   RAM               : {node.get_ram()}")
        print(f"   Disk              : {node.get_disk()}")
        print(f"   Image             : {node.get_image()}")
        print(f"   Image Type        : {node.get_image_type()}")
        print(f"   Host              : {node.get_host()}")
        print(f"   Site              : {node.get_site()}")
        print(f"   Management IP     : {node.get_management_ip()}")
        print(f"   Reservation ID    : {node.get_reservation_id()}")
        print(f"   Reservation State : {node.get_reservation_state()}")
        print(f"   SSH Command       : {node.get_ssh_command()}")
        print(f"   Components        :  ")
        for component in node.get_components():
            print(f"      Name             : {component.get_name()}")
            print(f"      Details          : {component.get_details()}")
            print(f"      Disk (G)         : {component.get_disk()}")
            print(f"      Units            : {component.get_unit()}")
            print(f"      PCI Address      : {component.get_pci_addr()}")
            print(f"      Model            : {component.get_model()}")
            print(f"      Type             : {component.get_type()}") 
        print(f"   Interfaces        :  ")
        for interface in node.get_interfaces():
            print(f"       Name                : {interface.get_name()}")
            print(f"           Bandwidth           : {interface.get_bandwidth()}")
            print(f"           VLAN                : {interface.get_vlan()}")            
    
except Exception as e:
    print(f"Fail: {e}")
    traceback.print_exc()

Node:
   Name              : s1
   Cores             : 2
   RAM               : 8
   Disk              : 100
   Image             : default_ubuntu_20
   Image Type        : qcow2
   Host              : max-w4.fabric-testbed.net
   Site              : MAX
   Management IP     : 63.239.135.106
   Reservation ID    : d5cdce3a-75e6-4d2b-b41d-e9b307b2c45f
   Reservation State : Active
   SSH Command       : ssh -i /Users/pruth/.ssh/id_rsa -J pruth@bastion-1.fabric-testbed.net ubuntu@63.239.135.106
   Components        :  
      Name             : s1-s1_local_nic
      Details          : Mellanox ConnectX-6 VPI MCX653 dual port 100Gbps
      Disk (G)         : 0
      Units            : 1
      PCI Address      : 0000:e2:02.5
      Model            : ConnectX-6
      Type             : SharedNIC
      Name             : s1-s1_switch_nic
      Details          : Mellanox ConnectX-5 Dual Port 10/25GbE
      Disk (G)         : 0
      Units            : 1
      PCI Address      : ['0000:a1:00.0

In [ ]:
import json

node = slice.get_node('s1')
print(f"node: {node.get_name()}")

stdout, stderr = node.execute("sudo ip -j route list")
pythonObj = json.loads(stdout)

#print(pythonObj)
management_dev = None
for i in pythonObj:
    if i['dst'] == 'default':
        management_dev = i['dev']

print(f"management_dev: {management_dev}")


stdout, stderr = node.execute("sudo ip -j addr list")
pythonObj = json.loads(stdout)
dataplane_devs = []
for i in pythonObj:
    if i['ifname'] != 'lo' and i['ifname'] !=  management_dev:
        dataplane_devs.append(i['ifname']) 
        
print(f"dataplane_devs: {dataplane_devs}")




In [ ]:
node = slice.get_node('s1')

print(node.get_dataplane_os_interfaces())
print(node.flush_all_os_interfaces())

stdout, stderr = node.execute("sudo ip addr list")
print(stdout)

In [ ]:
node.add_vlan_os_interface(os_iface='ens7',  vlan='42', ip='192.168.42.42', cidr='24', mtu='9000')
#node.set_ip_os_interface(os_iface='ens7', ip='192.168.42.42', cidr='24', mtu='9000')

stdout, stderr = node.execute("sudo ip addr list")
print(stdout)

In [ ]:
print(node.ping_test(dst_ip='192.168.1111.100'))
#node.set_ip_os_interface(os_iface='ens7', ip='192.168.42.42', cidr='24', mtu='9000')

stdout, stderr = node.execute("sudo ip addr list")
print(stdout)

In [ ]:
node.flush_os_interface('ens7.42')
node.remove_vlan_os_interface(os_iface='ens7',  vlan='42')

stdout, stderr = node.execute("sudo ip addr list")
print(stdout)

In [ ]:
node = slice.get_node('s2')

management_os_iface = node.get_management_os_interface()
print(f"management_os_iface: {management_os_iface}")

stdout, stderr = node.execute("sudo ip -j addr list")
stdout_json = json.loads(stdout)
dataplane_devs = []
for i in stdout_json:
    if i['ifname'] == management_os_iface or i['ifname'] == 'lo':
        stdout_json.remove(i)
        continue
    
    #If iface is vlan linked to base iface
    if 'link' in i.keys():
        print(f"remove_vlan_os_interface: {i['ifname']}, {i['link']}")
        node.remove_vlan_os_interface(os_iface=i['ifname'])
     
    
    
    #if i['ifname'] != 'lo' and i['ifname'] !=  management_dev:
    #    dataplane_devs.append(i['ifname'])


In [ ]:
node = slice.get_node('s2')
#node.remove_all_vlan_os_interfaces()
node.clear_all_ifaces()

In [6]:
for net in slice.get_l2networks():
    #net = slice.get_l2network('net_s1_s2')
    print(f"XXXX NET XXXXX: {net.get_name()}")
    ifaces = net.get_interfaces()

    #target iface/node
    target_iface =  ifaces.pop()
    target_node = target_iface.get_node()
    target_os_ifaces = target_node.get_dataplane_os_interfaces()
    target_node.clear_all_ifaces()
    
    print(f"{target_node.get_ssh_command()}")

    for os_iface in target_os_ifaces:
        iface_num=target_os_ifaces.index(os_iface)+1
        target_node.set_ip_os_interface(os_iface=os_iface,
                                          vlan=target_iface.get_vlan(),
                                          ip=f'192.168.{iface_num}.1',
                                          cidr = '24'
                                         )

    print(f"target_iface: {target_iface.get_name()}")
    print(f"target_iface.get_vlan(): {target_iface.get_vlan()}")
    print(f"target_node: {target_node.get_name()}")
    print(f"target_os_ifaces: {target_os_ifaces}")
    
    for iface in ifaces:
        node = iface.get_node()
        node.clear_all_ifaces()
        node_os_ifaces = node.get_dataplane_os_interfaces()

        
        print(f"test_node: {node.get_name()}")
        print(f"test_iface: {iface.get_name()}")
        print(f"node_os_ifaces: {node_os_ifaces}")
        print(f"iface.get_vlan(): {iface.get_vlan()}")
        print(f"{node.get_ssh_command()}")
        
        for node_os_iface in node_os_ifaces:
            for net_num in ['1','2','3']:
                dst_ip=f'192.168.{net_num}.1'

                ip=f'192.168.{net_num}.2'
                
                #set interface 
                node.set_ip_os_interface(os_iface=node_os_iface, 
                                         vlan=iface.get_vlan(), 
                                         ip=ip, 
                                         cidr='24')
                
                
                #ping test
                print(f"ping test {node.get_name()}:{node_os_iface} ->  - {ip} to {dst_ip}")
                print(f"Ping test output: {node.ping_test(dst_ip)}")
                
                #import time
                #time.sleep(10)
                
                #clean up interface
                
                if iface.get_vlan() == None:
                    node.flush_os_interface(node_os_iface) 
                else:
                    node.remove_vlan_os_interface(os_iface=f"{node_os_iface}.{iface.get_vlan()}")


        
        #if ping success record which os_iface (target and node)
        
            
     
    target_node.clear_all_ifaces()

    

XXXX NET XXXXX: net_s1_s2
management_os_iface: ens3
ssh -i /Users/pruth/.ssh/id_rsa -J pruth@bastion-1.fabric-testbed.net ubuntu@129.114.110.98
target_iface: s2-s2_switch_nic-p2
target_iface.get_vlan(): 1000
target_node: s2
target_os_ifaces: ['ens7', 'ens8', 'ens9']
management_os_iface: ens3
test_node: s1
test_iface: s1-s1_switch_nic-p2
node_os_ifaces: ['ens7', 'ens8', 'ens9']
iface.get_vlan(): 1000
ssh -i /Users/pruth/.ssh/id_rsa -J pruth@bastion-1.fabric-testbed.net ubuntu@63.239.135.106
ping test s1:ens7 ->  - 192.168.1.2 to 192.168.1.1
Ping test output: False
ping test s1:ens7 ->  - 192.168.2.2 to 192.168.2.1
Ping test output: False
ping test s1:ens7 ->  - 192.168.3.2 to 192.168.3.1
Ping test output: False
ping test s1:ens8 ->  - 192.168.1.2 to 192.168.1.1
Ping test output: False
ping test s1:ens8 ->  - 192.168.2.2 to 192.168.2.1
Ping test output: True
ping test s1:ens8 ->  - 192.168.3.2 to 192.168.3.1
Ping test output: False
ping test s1:ens9 ->  - 192.168.1.2 to 192.168.1.1
Ping 

In [ ]:
for node in slice.get_nodes():
    #node.clear_all_ifaces()
    print(f"node {node.get_name()}: {node.ping_test('192.168.3.100')}")

In [ ]:
node = slice.get_node('s1')

command = f'hostname'
stdout, stderr = node.execute(command)
print(stdout)

dst_ip='192.168.3.1'
command = f'ping -c 3 192.168.3.2 2>&1 > /dev/null && echo Success'
stdout, stderr = node.execute(command)
print(f"stdout: {stdout}")
print(f"stderr: {stderr}")
if stdout.replace("\n","") == 'Success':
    print('True')
else:
    print('False')

In [ ]:
for net in slice.get_l2networks():
    print(f"---- NET: {net.get_name()}")
    ifaces = net.get_interfaces()
    
    #target iface/node
    target_iface =  ifaces.pop()
    target_node = target_iface.get_node()
    target_os_ifaces = target_node.get_dataplane_os_interfaces()
    target_node.flush_all_os_interfaces()
    print(f"{target_node.get_ssh_command()}")
    
    

### Find NIC Mappings

In [ ]:
flush_all_dataplane_ips()

iface_map = {}
net = 'net_s1_s2'
print (f'net: {net}')
ifaces = find_nic_mapping(net, [experiment_topology.nodes[s1_name],experiment_topology.nodes[s2_name]] )
iface_map[net] = ifaces

flush_all_dataplane_ips()


net = 'net_s2_s3'
print (f'net: {net}')
ifaces = find_nic_mapping(net, [experiment_topology.nodes[s2_name],experiment_topology.nodes[s3_name]] )
iface_map[net] = ifaces
flush_all_dataplane_ips()


net = 'net_s3_s1'
print (f'net: {net}')
ifaces = find_nic_mapping(net, [experiment_topology.nodes[s3_name],experiment_topology.nodes[s1_name]] )
iface_map[net] = ifaces

flush_all_dataplane_ips()


net = 'net_s1_h1'
print (f'net: {net}')
ifaces = find_nic_mapping(net, [experiment_topology.nodes[s1_name],experiment_topology.nodes[h1_name]] )
iface_map[net] = ifaces

flush_all_dataplane_ips()


net = 'net_s2_h2'
print (f'net: {net}')
ifaces = find_nic_mapping(net, [experiment_topology.nodes[s2_name],experiment_topology.nodes[h2_name]] )
iface_map[net] = ifaces

flush_all_dataplane_ips()


net = 'net_s3_h3'
print (f'net: {net}')
ifaces = find_nic_mapping(net, [experiment_topology.nodes[s3_name],experiment_topology.nodes[h3_name]] )
iface_map[net] = ifaces

flush_all_dataplane_ips()


print (f'{iface_map}')



## Configure Nodes


In [ ]:
node_num=1
for host_name in [h1_name, h2_name, h3_name]:
    #Configure h1
    print(f'Configuring {host_name}')
    host = experiment_topology.nodes[host_name]
    host_ip = '10.0.'+str(node_num)+'.'+str(node_num)
    host_cidr = '24'
    host_management_ip = str(host.management_ip)
    print(f'Management IP: {host_management_ip}')
    file_attributes = upload_file(username, host, 'scripts/host_set_dataplane_ip.py','host_set_dataplane_ip.py')
    print("file_attributes: {}".format(file_attributes))
        
    stdout = execute_script(username, host, 'sudo python3 host_set_dataplane_ip.py {} {}'.format(host_ip,host_cidr) )
    print("stdout: {}".format(stdout))
    
    stdout = execute_script(username, host, 'sudo apt-get update -qq && sudo apt-get install -qq -y python3-scapy && git clone https://github.com/p4lang/tutorials.git')
    print("stdout: {}".format(stdout))
    
    node_num = node_num +1


## Configure Switches

Use ssh to configure the ifaces on the switches. This step requires testing the interfaces to figure out which interface is connected to which network.


#### Setup P4 Docker



In [ ]:
def get_iface_list_XXX(vm_name):
    iface_str=""
    for net_name,iface_dict in iface_map.items():
        if vm_name in iface_dict.keys():
            iface_str = iface_str + " " + str(iface_dict[vm_name])
            
    return iface_str


def get_iface_list(vm_name):
    iface_str=""
    if vm_name == s1_name:
        iface_str=f"{iface_map['net_s1_h1'][vm_name]} {iface_map['net_s1_s2'][vm_name]} {iface_map['net_s3_s1'][vm_name]}"
    elif vm_name == s2_name:
        iface_str=f"{iface_map['net_s2_h2'][vm_name]} {iface_map['net_s1_s2'][vm_name]} {iface_map['net_s2_s3'][vm_name]}"
    elif vm_name == s3_name:
        iface_str=f"{iface_map['net_s3_h3'][vm_name]} {iface_map['net_s3_s1'][vm_name]} {iface_map['net_s2_s3'][vm_name]}"
    else:
        print(f'Unknown vm: {vm_name}')
            
    return iface_str


for switch_name in [s1_name, s2_name, s3_name]:

    switch_node = experiment_topology.nodes[switch_name]
    management_ip_switch = str(switch_node.management_ip)
    print("Swtitch Name        : {}".format(switch_node.name))
    print("Management IP    : {}".format(management_ip_switch))

    iface_str = get_iface_list(switch_name)
    print(f"iface_str: {iface_str}")
    
    file_attributes = upload_file(username, switch_node, 'scripts/router_setup_p4_bmv2_container.sh','router_setup_p4_bmv2_container.sh')
    #print("file_attributes: {}".format(file_attributes))

    stdout = execute_script(username, switch_node, f"chmod +x router_setup_p4_bmv2_container.sh && sudo sh -c './router_setup_p4_bmv2_container.sh {iface_str}  > /tmp/script.log 2>&1'")
    #print("stdout: {}".format(stdout))
    
    

### Confgure P4 Switch Tables

Edit sX_commands.txt to change the values

In [ ]:
for switch_name in [s1_name, s2_name, s3_name]:
    switch_node = experiment_topology.nodes[switch_name]
    management_ip_switch = str(switch_node.management_ip)
    print("Swtitch Name        : {}".format(switch_node.name))
    print("Management IP    : {}".format(management_ip_switch))
    
    
    #Configure P4 Tables
    cmd_file=f'{switch_name}_commands.txt'
    print(cmd_file)
    file_attributes = upload_file(username, switch_node, f'scripts/{cmd_file}',cmd_file)
    print("file_attributes: {}".format(file_attributes))

    #stdout = execute_script(username, switch_node, f"sudo sh -c 'cat {cmd_file} | docker exec -it fabric_p4 simple_switch_CLI  > /tmp/script.log 2>&1'")
    stdout = execute_script(username, switch_node, f"sudo sh -c 'cat {cmd_file} | docker exec -i fabric_p4 simple_switch_CLI'")
    print("stdout: {}".format(stdout))

### Run the P4 Swtiches

In [ ]:
#for switch_name in [s1_name, s2_name, s3_name]:
for switch_name in [s1_name]:
    
    switch_node = experiment_topology.nodes[switch_name]
    management_ip_switch = str(switch_node.management_ip)
    print("Swtitch Name        : {}".format(switch_node.name))
    print("Management IP    : {}".format(management_ip_switch))

    iface_str = get_iface_list(switch_name)
    print(f"iface_str: {iface_str}")
    
    #file_attributes = upload_file(username, switch_node, 'router_setup_p4_bmv2_container.sh','router_setup_p4_bmv2_container.sh')
    #print("file_attributes: {}".format(file_attributes))

    #stdout = execute_script(username, switch_node, f"chmod +x router_setup_p4_bmv2_container.sh && sudo sh -c './router_setup_p4_bmv2_container.sh {iface_str}  > /tmp/script.log 2>&1'")
    #print("stdout: {}".format(stdout))
    
    
    #stdout = execute_script(username, switch_node, f"sudo sh -c 'docker exec -w /root/tutorials/exercises/basic_tunnel fabric_p4 cp solution/basic_tunnel.p4 basic_tunnel.p4'")
    #print("stdout: {}".format(stdout))
    #stdout = execute_script(username, switch_node, f"sudo sh -c 'docker exec -w /root/tutorials/exercises/basic_tunnel fabric_p4 p4c --p4runtime-files basic_tunnel.txt --target bmv2 --arch v1model basic_tunnel.p4'")
    #print("stdout: {}".format(stdout))
    #stdout = execute_script(username, switch_node, f"sudo sh -c 'docker exec -d -w /root/tutorials/exercises/basic_tunnel fabric_p4 simple_switch --interface 0@ens7 --interface 1@ens8 --interface 2@ens9 basic_tunnel.json'")
    #print("stdout: {}".format(stdout))
    
    
    #echo "echo This is how we pipe to docker exec" | sudo docker exec --interactive CONTAINER_NAME /bin/bash - 


    #stdout = execute_script(username, switch_node, f"sudo sh -c 'echo table_set_default myTunnel_exact drop | docker exec -i -w /root/tutorials/exercises/basic_tunnel fabric_p4 /usr/local/bin/simple_switch_CLI --thrift-port 9090'")
    #print("stdout: {}".format(stdout)) 
    #stdout = execute_script(username, switch_node, f'echo "table_add myTunnel_exact myTunnel_forward 1 \=\> 1" | sudo sh -c docker exec -i -w /root/tutorials/exercises/basic_tunnel fabric_p4 /usr/local/bin/simple_switch_CLI --thrift-port 9090')
    #print("stdout: {}".format(stdout))
    stdout = execute_script(username, switch_node, f"sudo sh -c 'docker exec -i -w /root/tutorials/exercises/basic_tunnel fabric_p4 echo \"table_add myTunnel_exact myTunnel_forward 1 => 1\" \\| /usr/local/bin/simple_switch_CLI --thrift-port 9090'")
    print("stdout: {}".format(stdout))
    #stdout = execute_script(username, switch_node, f"sudo sh -c 'docker exec -w /root/tutorials/exercises/basic_tunnel fabric_p4 echo \'table_add myTunnel_exact myTunnel_forward 2 => 2\' \| /usr/local/bin/simple_switch_CLI --thrift-port 9090'")
    #print("stdout: {}".format(stdout))
    #stdout = execute_script(username, switch_node, f"sudo sh -c 'docker exec -w /root/tutorials/exercises/basic_tunnel fabric_p4 echo \'table_add myTunnel_exact myTunnel_forward 3 => 3\' \| /usr/local/bin/simple_switch_CLI --thrift-port 9090'")
    #print("stdout: {}".format(stdout))

## Delete Slice

In [ ]:
return_status, result = slice_manager.delete(slice_object=slice)

print("Response Status {}".format(return_status))
print("Response received {}".format(result))